# Exploratory Data Analysis (EDA)

In this notebook, we will perform exploratory data analysis on the Tunisian stock market data to gain insights and identify patterns.


In [1]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [2]:
# Reading the data
data = pd.read_csv('./data/cleaned_weekly_stock_market.csv')

# Display the first few rows of the dataset
data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume
0,SOTUMAG,2014-06-16,1.76,1.80,1.73,1.73,44315.0
1,SOTUMAG,2014-06-23,1.78,1.82,1.77,1.82,9551.0
2,SOTUMAG,2014-06-30,1.81,1.82,1.81,1.81,2401.0
3,SOTUMAG,2014-07-07,1.78,1.82,1.78,1.80,10341.0
4,SOTUMAG,2014-07-14,1.83,1.83,1.76,1.81,27480.0


## Summary Statistics

We will start by generating summary statistics for the dataset to understand its basic properties.


In [3]:
# Summary statistics
summary_stats = data.describe()
summary_stats


,openingPrice,highestPrice,lowestPrice,closingPrice,volume
count,38631.000000,38631.000000,38631.000000,38631.000000,3.863100e+04
mean,106.641003,107.559123,105.715594,106.777993,2.821250e+05
std,791.642044,797.612176,785.735260,792.626816,2.504922e+06
min,0.110000,0.110000,0.110000,0.110000,0.000000e+00
25%,2.930000,2.990000,2.850000,2.930000,1.610000e+03
50%,6.690000,6.800000,6.530000,6.700000,1.003900e+04
75%,14.705000,14.910000,14.470000,14.710000,3.901750e+04
max,9592.970000,9670.220000,9578.670000,9663.860000,1.124223e+08


In [4]:
data.shape

(38631, 7)

## Companies Stock Prices

we will visualize the stock prices of the companies in the dataset to identify patterns and trends.

In [5]:
data["companyName"].unique()

array(['SOTUMAG', 'TUNISAIR', 'STEQ', 'BANQUE ATTIJARI DE TUNIS',
       'HANNIBAL LEASE', 'AMI', 'CELLCOM', 'SIPHAT', 'TUNISIE VALEURS',
       'POULINA GROUP HOLDING', 'PLACEMENTS DE TUNISIE - SICAF', 'SIAME',
       'ONE TECH', 'TUNISIE LEASING & FACTORING', 'SOPAT',
       'ASSURANCES MAGHREBIA', 'WIFAK INT BANK', 'MAGASIN GENERAL',
       'SERVICOM', 'MIP', 'SOTUVER', 'UNIMED', 'SOTIPAPIER',
       'MEUBLES INTERIEURS', 'ESSOUKNA', 'STB BANK', 'UIB',
       'MAGHREBIA VIE', 'AMEN BANK', 'BH ASSURANCE',
       'SOCIETE CHIMIQUE ALKIMIA', 'BH BANK', 'CEREALIS', 'SOTRAPIL',
       'ATTIJARI LEASING', 'CITY CARS', 'MONOPRIX', 'CARTHAGE CEMENT',
       'ICF', 'BANQUE NATIONALE AGRICOLE', 'TPR', 'ELBENE', 'SOTEMAIL',
       'BH LEASING', 'ARAB TUNISIAN BANK', 'DELICE HOLDING', 'SOTETEL',
       'ARTES', 'SANIMED', 'TUNISIAN BOND INDEX', 'UADH', 'TAWASOL',
       'SMART TUNISIE', 'ASSAD', 'SYPHAX AIRLINES', 'TELNET HOLDING',
       'TUNINVEST - SICAR', 'SFBT', 'GIF FILTER', 'UBCI', "LAND

### SOTUMAG Stock Prices

In [40]:
if 'date' not in data.columns or 'closingPrice' not in data.columns:
    raise ValueError("The DataFrame does not have the required columns 'date' and 'closingPrice'")

data['date'] = pd.to_datetime(data['date'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data['date'],
    y=data['closingPrice'],
    mode='markers',
    name='Closing Price',
    hovertemplate='%{x|%Y-%m-%d} - %{y:.2f}<extra></extra>',
    marker=dict(
        size=3,
    )
))

fig.update_layout(
    title='Stock Market Closing Prices of SOTUMAG',
    xaxis_title='Date',
    yaxis_title='Closing Price',
    hovermode='y unified',
    template='plotly_white',
    showlegend=False
    
)
fig.update_yaxes(tickformat=".0f") 
fig.show()